## Proyecto Datos estadisticos de Bancos BCParaguay; 
Data Engineer:  Rodrigo Figueredo 
Objetivo: se necesita un proceso consistente de extraccion de datos para cargarlo en un Datawarehouse en SQL, el proceso deberá correr de forma mensual y que contenga los datos consistente y evitar a manipulación manual del excel origen.

Antes que nada debemos monitorear, depurar y auditar cada paso. 


In [10]:

# Configuración logging
# Primero, configuramos el logger al inicio del script:
import os
import logging
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime
import logging
import pyodbc
import re
import unidecode

# Obtener la ruta actual
ruta_actual = os.getcwd()
# Configurar logging
logging.basicConfig(
    filename=f'{ruta_actual}/etl_proceso.log',       # Archivo de log
    level=logging.INFO,               # Nivel de log: DEBUG, INFO, WARNING, ERROR, CRITICAL
    format='%(asctime)s - %(levelname)s - %(message)s',
    filemode='w'                      # 'w' sobrescribe cada vez, 'a' acumula
)

logger = logging.getLogger()
print('Se inicializa parametros')

Se inicializa parametros


🧩1. Extracción (Extract)

In [11]:
# Obtener ruta actual
ruta_actual = os.getcwd()
logger.info(f"Ruta actual de ejecución: {ruta_actual}")

# Ruta del archivo Excel
excel_file = 'C:/Users/rfigu/Documents/Python Scripts/BoletinBCP/1.1 Tablas Boletín Bancos Ago25 v2.xlsx'

# Extracción de hojas
try:
    logger.info(f"Iniciando lectura del archivo Excel: {excel_file}")
    sheets_dict = pd.read_excel(excel_file, sheet_name=None)
    logger.info(f"Lectura exitosa. Hojas encontradas: {list(sheets_dict.keys())}")
    
    # Acceder a cada hoja como DataFrame
    for sheet_name, df in sheets_dict.items():
        logger.info(f"Hoja: {sheet_name}, Filas: {len(df)}")
        print(f"Hoja: {sheet_name}, Filas: {len(df)}")

except Exception as e:
    logger.error(f"Error al leer el archivo Excel: {e}")
    print(f"Error: {e}")

Hoja: EEFF, Filas: 225891
Hoja: Ratios, Filas: 83353
Hoja: Carteras, Filas: 50290
Hoja: Credito Sector, Filas: 44007
Hoja: Credito Actividad, Filas: 9415
Hoja: Categoría Creditos, Filas: 14617
Hoja: TC, Filas: 3898
Hoja: Canales & Person, Filas: 13326
Hoja: Inhab, Filas: 7478


1.1 EDA - Exploratory Data Analysis

In [12]:
## Vista previa de los datos la hoja
sheets_dict.get('Carteras').tail()

,Fecha,Codigo Entidad,Codigo Cuenta,Codigo Moneda,Importe
50285,2025-08-31,1047,Cartera Vencida,6900,48115.510679
50286,2025-08-31,1047,Medidas transitorias,6200,53086.044503
50287,2025-08-31,1047,Medidas transitorias,6900,12081.982817
50288,2025-08-31,1047,Tarjetas de Crédito,6900,28516.548051
50289,2025-08-31,1047,Medida Excepcional COVID 19 - Vencida,6900,199.467029


In [13]:
## Vista previa de los tipos de datos la hoja
sheets_dict.get('TC').dtypes

Fecha             datetime64[ns]
Codigo Entidad             int64
Clasificación             object
Total                    float64
dtype: object

🔄 2. Transformación (Transform)

Aquí puedes aplicar limpieza, normalización, validación, etc. Ejemplo:


In [14]:
# Función de transformación con logging
def transforBol(df, nombre_hoja):
    try:
        logger.info(f"Iniciando transformación de hoja: {nombre_hoja}")

        # Eliminar filas vacías
        df = df.dropna(how='all')
        logger.info(f"Filas vacías eliminadas en hoja: {nombre_hoja}")

        # Renombrar columnas
        columnas_originales = df.columns.tolist()
        df.columns = [col.strip().lower().replace(' ', '_') for col in df.columns]
        logger.info(f"Columnas renombradas en hoja: {nombre_hoja} — Originales: {columnas_originales} → Nuevas: {df.columns.tolist()}")

        # Convertir fechas
        if 'fecha' in df.columns:
            df['fecha'] = pd.to_datetime(df['fecha'], errors='coerce')
            errores_fecha = df['fecha'].isna().sum()
            logger.info(f"Columna 'fecha' convertida en hoja: {nombre_hoja}. Errores de conversión: {errores_fecha}")
        else:
            logger.warning(f"No se encontró columna 'fecha' en hoja: {nombre_hoja}")

        logger.info(f"Transformación completada para hoja: {nombre_hoja}")
        return df

    except Exception as e:
        logger.error(f"Error al transformar hoja {nombre_hoja}: {e}")
        raise

# Aplicar transformación a cada hoja
datos_transformados = {}

for sheet, df in sheets_dict.items():
    datos_transformados[sheet] = transforBol(df, sheet)

In [15]:
datos_transformados.get('Credito Sector').head(3)

,fecha,codigo_entidad,codigo_moneda,actividad_destino_vs2,cartera_vencida,cartera_vigente
0,2016-01-31,1002,6200,AGRICULTURA,2416.278988,152923.648836
1,2016-01-31,1002,6200,SECTOR FINANCIERO,0.000000,83374.169137
2,2016-01-31,1002,6200,VIVIENDA,0.000000,2033.573859


📥 3. Carga (Load)

In [21]:
# Consulta de prueba
query = "SELECT * FROM dbo.TC"
df_query = pd.read_sql(query, connP)
df_query.tail()

C:\Users\rfigu\AppData\Local\Temp\ipykernel_22592\2928162568.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_query = pd.read_sql(query, connP)


,fecha,codigo_entidad,clasificación,total,fecha_carga
3893,2025-08-31,1045,Saldo,4.997059e+04,2025-10-11 13:15:49.997
3894,2025-08-31,1046,Cantidad,1.081194e+06,2025-10-11 13:15:49.997
3895,2025-08-31,1046,Saldo,6.626615e+05,2025-10-11 13:15:49.997
3896,2025-08-31,1047,Cantidad,5.703000e+03,2025-10-11 13:15:49.997
3897,2025-08-31,1047,Saldo,2.851655e+04,2025-10-11 13:15:49.997


In [17]:
# Parámetros de conexión
server = 'ASUSTUF\SQL22'  # Doble barra para escapar correctamente
database = 'BolBcp'

try:
    logger.info(f"Intentando conectar a SQL Server: {server}, Base de datos: {database}")
    
    connP = pyodbc.connect(
        'DRIVER={ODBC Driver 17 for SQL Server};'
        f'SERVER={server};'
        f'DATABASE={database};'
        'Trusted_Connection=yes;'
    )
    
    logger.info("Conexión a SQL Server establecida exitosamente.")
    print('Conexión exitosa')

except pyodbc.Error as e:
    logger.error(f"Error al conectar a la base de datos: {str(e)}")
    print(f'Error al conectar a la base de datos: {str(e)}')


Conexión exitosa


In [18]:
# Renombrar claves del diccionario
datoslimpios = {
    re.sub(r"[^\w]", "", unidecode.unidecode(k.strip())): v
    for k, v in datos_transformados.items()
}

# Recorrer el diccionario con claves limpias
for nombre_tabla, df in datoslimpios.items():
    print(nombre_tabla)


EEFF
Ratios
Carteras
CreditoSector
CreditoActividad
CategoriaCreditos
TC
CanalesPerson
Inhab


In [28]:
# SQLAlchemy Parámetros de conexión
server = 'ASUSTUF\SQL22'
database = 'BolBcp'
driver = 'ODBC Driver 17 for SQL Server'

# Crear cadena de conexión para SQLAlchemy
# conexion_str = f"mssql+pyodbc://@{server}/{database}?driver={driver}&trusted_connection=yes"
conexion_str = f"mssql+pyodbc://@{server}/{database}?driver={driver}&trusted_connection=yes"

try:
    engine = create_engine(conexion_str)
    print(f"✅ conexion exitosa")
    logger.info(f"Conexión a SQL Server establecida: {server}, Base de datos: {database}")
except Exception as e:
    print(f"❌ fallo en la conexion")
    logger.error(f"Error al crear engine SQLAlchemy: {e}")
    raise

✅ conexion exitosa


In [29]:
# Cargar cada hoja transformada como tabla
for nombre_tabla, df in datoslimpios.items():
    try:
        logger.info(f"Iniciando carga de hoja: {nombre_tabla}")

        # Eliminar filas completamente vacías
        df = df.dropna(how='all')
        # Agregar columna de fecha y hora de carga
        df['fecha_carga'] = datetime.now()

        # Insertar usando pandas y SQLAlchemy
        df.to_sql(name=nombre_tabla, con=engine, if_exists='replace', index=False)

        logger.info(f"Hoja '{nombre_tabla}' cargada exitosamente con {len(df)} filas.")
        print(f"✅ Hoja '{nombre_tabla}' cargada exitosamente con {len(df)} filas.")

    except Exception as e:
        logger.error(f"Error al cargar hoja '{nombre_tabla}': {e}")
        print(f"❌ Error al cargar hoja '{nombre_tabla}': {e}")

✅ Hoja 'EEFF' cargada exitosamente con 225891 filas.
✅ Hoja 'Ratios' cargada exitosamente con 83353 filas.
✅ Hoja 'Carteras' cargada exitosamente con 50290 filas.
✅ Hoja 'CreditoSector' cargada exitosamente con 44007 filas.
✅ Hoja 'CreditoActividad' cargada exitosamente con 9415 filas.
✅ Hoja 'CategoriaCreditos' cargada exitosamente con 14617 filas.
✅ Hoja 'TC' cargada exitosamente con 3898 filas.
✅ Hoja 'CanalesPerson' cargada exitosamente con 13326 filas.
✅ Hoja 'Inhab' cargada exitosamente con 7478 filas.


Leer por nombre de tabla (si el Excel tiene tablas definidas)


In [ ]:
# ## conexion a la Base datos SQL Server v2022
# path = 'C:/Users/rfigu/Documents/Python Scripts/BoletinBCP/Dimensiones y referencias.xlsx'


# dfDepara = pd.read_excel(path, sheet_name="entidad")
# logger.info(f"Archivo cargado con {len(dfDepara)} registros")

# dfDepara.head()

In [ ]:
# dfDepara.to_sql(name='DeParabanco', con=conexion_str, if_exists='replace',index=False)